In [35]:
# FOLDER = '../data/olivia/Healthy'
FOLDER = '../data/olivia/Flat feet'
EXTENSION = '.obj'
# EXTENSION = '.stl'
# OUTPUT_FOLDER = 'output/olivia/Healthy'
OUTPUT_FOLDER = 'output/olivia/Flat feet'

In [36]:
import sys
sys.path.append('..')

In [37]:
import os

files = os.listdir(FOLDER)
files = [os.path.join(FOLDER, f) for f in files if EXTENSION in f]
files.sort()
files

['../data/olivia/Flat feet/AM002R.obj']

## Example run

In [30]:
import pandas as pd

file = files[0]

# load labelling
lmrk = pd.concat([
    pd.read_pickle(file.replace(EXTENSION, '.pkl')),
    pd.read_pickle(file.replace(EXTENSION, '-extra-udmc.pkl')),
])
lmrk

coord,x,y,z
landmark,,,
P1,-67.433754,-59.069650,-4.445435
P10,143.191633,-63.485578,-8.482214
P11,96.316931,-102.094039,-68.546837
P12,105.704739,-43.645970,-64.195482
P2,-54.321493,-98.332041,-2.974623
P3,-17.066654,-15.908056,-8.567077
P4,-15.298339,-100.373671,-13.646069
P5,13.118212,-16.114940,-5.127648
P6,41.250061,-75.062300,-57.208651


In [18]:
import pyvista as pv
from mesh4d.analyse import crave
from measure import frame, visual

# local frame
mesh = crave.fix_pvmesh_disconnect(pv.read(file), lmrk.values)
axes_frame, origin = frame.estimate_foot_frame(mesh, lmrk)
mesh_clip = frame.foot_clip(mesh, lmrk)
mesh_local = frame.foot2local(mesh_clip, axes_frame, origin)
lmrk_local = frame.df2local(lmrk, axes_frame, origin)

# visual check
visual.plot_axes(origin, axes_frame, mesh_clip)

Widget(value='<iframe src="http://localhost:64987/index.html?ui=P_0x164cbb1c0_5&reconnect=auto" class="pyvista…

In [19]:
lmrk_local

coord,x,y,z
landmark,,,
P1,107.762781,-1.332880,4.478260
P10,-120.428164,-3.186499,5.889827
P11,-67.779877,-39.195876,63.614944
P12,-78.036585,21.409069,49.043421
P2,95.959077,-38.739356,2.490667
P3,58.067967,40.383067,9.251058
P4,56.144391,-40.924617,13.593470
P5,33.052191,41.009551,12.636770
P6,13.029236,-6.425653,43.548940


In [31]:
from measure.metric import height, dist, dist_along_axis

def al(
        lmkr_local: pd.DataFrame,
    ):
    """arch length"""
    return dist(lmkr_local, 'MLA-AP', 'MLA-PP')

def ab(
        lmkr_local: pd.DataFrame,
    ):
    """arch breadth"""
    return dist(lmkr_local, 'LMB', 'MMB')

def ad(
        lmkr_local: pd.DataFrame,
    ):
    """arch depth"""
    return dist_along_axis(lmkr_local, 'P4', 'MMB', 'x')

def ah(
        lmkr_local: pd.DataFrame,
    ):
    """arch height"""
    return height(lmkr_local, 'AA')

In [32]:
# metrics
results = {
    'file': ['description', str(file)],
    'AL': ['arch length (mm)', al(lmrk_local)],
    'AB': ['arch breadth (mm)', ab(lmrk_local)],
    'AD': ['arch depth (mm)', ad(lmrk_local)],
    'AH': ['arch height (mm)', ah(lmrk_local)],
}

df_extra = pd.DataFrame(results)
df_extra

,file,AL,AB,AD,AH
0,description,arch length (mm),arch breadth (mm),arch depth (mm),arch height (mm)
1,../data/olivia/Healthy/AM004L.stl,106.121039,43.842147,63.48154,13.619875


### Visualization

In [22]:
settings = {
    'AL': ['MLA-AP', 'MLA-PP'],
    'AB': ['LMB', 'MMB'],
}

for name, [landmark1, landmark2] in settings.items():
    scene = pv.Plotter()
    scene.add_mesh(mesh_local, opacity=0.1)
    visual.plot_dist(scene, lmrk_local, landmark1, landmark2, name=name)
    scene.show()
    scene.screenshot(f'{OUTPUT_FOLDER}/{name}.png')

Widget(value='<iframe src="http://localhost:64987/index.html?ui=P_0x111b03e80_6&reconnect=auto" class="pyvista…

Widget(value='<iframe src="http://localhost:64987/index.html?ui=P_0x15dc8f910_7&reconnect=auto" class="pyvista…

In [23]:
settings = {
    'AD': ['P4', 'MMB', 'x'],
}

for name, [landmark1, landmark2, axis] in settings.items():
    scene = pv.Plotter()
    scene.add_mesh(mesh_local, opacity=0.1)
    visual.plot_dist_along_axis(scene, lmrk_local, landmark1, landmark2, axis, name=name)
    scene.show()
    scene.screenshot(f'{OUTPUT_FOLDER}/{name}.png')

Widget(value='<iframe src="http://localhost:64987/index.html?ui=P_0x1665e1d80_8&reconnect=auto" class="pyvista…

In [24]:
settings = {
    'AH': 'AA',
}

for name, landmark in settings.items():
    scene = pv.Plotter()
    scene.add_mesh(mesh_local, opacity=0.1)
    visual.plot_height(scene, lmrk_local, landmark, name=name)
    scene.show()
    scene.screenshot(f'{OUTPUT_FOLDER}/{name}.png')

Widget(value='<iframe src="http://localhost:64987/index.html?ui=P_0x1665e3e50_9&reconnect=auto" class="pyvista…

## Run all

In [38]:
os.path.join(
    OUTPUT_FOLDER,
    os.path.basename(file).replace(EXTENSION, '-extra.csv'),
    )

'output/olivia/Flat feet/BM025L.stl'

In [39]:
from tqdm import tqdm
from measure import metric

for file in tqdm(files):
    # load labelling
    lmrk = pd.concat([
        pd.read_pickle(file.replace(EXTENSION, '.pkl')),
        pd.read_pickle(file.replace(EXTENSION, '-extra-udmc.pkl')),
    ])

    # local frame
    mesh = crave.fix_pvmesh_disconnect(pv.read(file), lmrk.values)
    axes_frame, origin = frame.estimate_foot_frame(mesh, lmrk)
    mesh_clip = frame.foot_clip(mesh, lmrk)
    mesh_local = frame.foot2local(mesh_clip, axes_frame, origin)
    lmrk_local = frame.df2local(lmrk, axes_frame, origin)

    # metrics
    results = {
        'file': ['description', str(file)],
        'AL': ['arch length (mm)', al(lmrk_local)],
        'AB': ['arch breadth (mm)', ab(lmrk_local)],
        'AD': ['arch depth (mm)', ad(lmrk_local)],
        'AH': ['arch height (mm)', ah(lmrk_local)],
    }

    df_extra = pd.DataFrame(results)
    df_extra = pd.DataFrame(results).set_index('file')

    output_path = os.path.join(
        OUTPUT_FOLDER,
        os.path.basename(file).replace(EXTENSION, '-extra.csv'),
        )
    df_extra.to_csv(output_path)

    # combine all results so far
    output_path = os.path.join(OUTPUT_FOLDER, 'measurements-extra.csv')
    df_all = metric.combine_measurement_csv(OUTPUT_FOLDER)
    df_all.to_csv(output_path)

100%|██████████| 1/1 [00:02<00:00,  2.35s/it]
